In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns',100)

In [2]:
data = pd.read_csv('XYZCorp_LendingData.csv',parse_dates=['issue_d'])

/usr/local/Cellar/ipython/7.7.0/libexec/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (17,45,53) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.head(3)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,default_ind
0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-11,n,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0,Jan-85,1,NaN,NaN,3,0,13648,83.7,9,f,0.0,0.0,5861.071414,5831.78,5000.00,861.07,0.0,0.00,0.00,Jan-15,171.62,NaN,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-11,n,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0,Apr-99,5,NaN,NaN,3,0,1687,9.4,4,f,0.0,0.0,1008.710000,1008.71,456.46,435.17,0.0,117.08,1.11,Apr-13,119.66,NaN,Sep-13,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-11,n,NaN,small_business,real estate business,606xx,IL,8.72,0,Nov-01,2,NaN,NaN,2,0,2956,98.5,10,f,0.0,0.0,3003.653644,3003.65,2400.00,603.65,0.0,0.00,0.00,Jun-14,649.91,NaN,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [4]:
nulls = dict(data.isnull().sum()/len(data)*100)
drops = []
for i in nulls:
    if nulls[i] > 65:
        drops.append(i)
drops

['desc',
 'mths_since_last_record',
 'mths_since_last_major_derog',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'open_acc_6m',
 'open_il_6m',
 'open_il_12m',
 'open_il_24m',
 'mths_since_rcnt_il',
 'total_bal_il',
 'il_util',
 'open_rv_12m',
 'open_rv_24m',
 'max_bal_bc',
 'all_util',
 'inq_fi',
 'total_cu_tl',
 'inq_last_12m']

In [5]:
data = data.drop(columns=drops)
data.isnull().sum()/len(data)*100

id                             0.000000
member_id                      0.000000
loan_amnt                      0.000000
funded_amnt                    0.000000
funded_amnt_inv                0.000000
term                           0.000000
int_rate                       0.000000
installment                    0.000000
grade                          0.000000
sub_grade                      0.000000
emp_title                      5.776261
emp_length                     5.030673
home_ownership                 0.000000
annual_inc                     0.000000
verification_status            0.000000
issue_d                        0.000000
pymnt_plan                     0.000000
purpose                        0.000000
title                          0.003855
zip_code                       0.000000
addr_state                     0.000000
dti                            0.000000
delinq_2yrs                    0.000000
earliest_cr_line               0.000000
inq_last_6mths                 0.000000


In [6]:
data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,default_ind
0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-11,n,credit_card,Computer,860xx,AZ,27.65,0,Jan-85,1,NaN,3,0,13648,83.7,9,f,0.0,0.0,5861.071414,5831.78,5000.00,861.07,0.00,0.00,0.00,Jan-15,171.62,NaN,Jan-16,0.0,1,INDIVIDUAL,0,NaN,NaN,NaN,0
1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-11,n,car,bike,309xx,GA,1.00,0,Apr-99,5,NaN,3,0,1687,9.4,4,f,0.0,0.0,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,NaN,Sep-13,0.0,1,INDIVIDUAL,0,NaN,NaN,NaN,1
2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-11,n,small_business,real estate business,606xx,IL,8.72,0,Nov-01,2,NaN,2,0,2956,98.5,10,f,0.0,0.0,3003.653644,3003.65,2400.00,603.65,0.00,0.00,0.00,Jun-14,649.91,NaN,Jan-16,0.0,1,INDIVIDUAL,0,NaN,NaN,NaN,0
3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-11,n,other,personel,917xx,CA,20.00,0,Feb-96,1,35.0,10,0,5598,21.0,37,f,0.0,0.0,12226.302210,12226.30,10000.00,2209.33,16.97,0.00,0.00,Jan-15,357.48,NaN,Jan-15,0.0,1,INDIVIDUAL,0,NaN,NaN,NaN,0
4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-11,n,other,Personal,972xx,OR,17.94,0,Jan-96,0,38.0,15,0,27783,53.9,38,f,766.9,766.9,3242.170000,3242.17,2233.10,1009.07,0.00,0.00,0.00,Jan-16,67.79,Feb-16,Jan-16,0.0,1,INDIVIDUAL,0,NaN,NaN,NaN,0


In [7]:
data.emp_title = data.emp_title.fillna('Not Collected')
data.emp_length = data.emp_length.fillna('0')
data.mths_since_last_delinq = data.mths_since_last_delinq.fillna('0')
data.revol_util = data.revol_util.fillna('0')
data.next_pymnt_d = data.next_pymnt_d.fillna('Not Collected')
data.last_pymnt_d = data.last_pymnt_d.fillna('Not Collected')
data.last_credit_pull_d = data.last_credit_pull_d.fillna('Not Collected')

In [8]:
data = data[data.tot_cur_bal.notnull()]
data.isnull().sum()

id                             0
member_id                      0
loan_amnt                      0
funded_amnt                    0
funded_amnt_inv                0
term                           0
int_rate                       0
installment                    0
grade                          0
sub_grade                      0
emp_title                      0
emp_length                     0
home_ownership                 0
annual_inc                     0
verification_status            0
issue_d                        0
pymnt_plan                     0
purpose                        0
title                         22
zip_code                       0
addr_state                     0
dti                            0
delinq_2yrs                    0
earliest_cr_line               0
inq_last_6mths                 0
mths_since_last_delinq         0
open_acc                       0
pub_rec                        0
revol_bal                      0
revol_util                     0
total_acc 

In [9]:
data = data.drop(columns=['last_pymnt_d','last_pymnt_amnt','last_credit_pull_d','next_pymnt_d','mths_since_last_delinq'])

In [10]:
data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,default_ind
39694,10149342,12000897,27050,27050,27050.0,36 months,10.99,885.46,B,B2,Team Leadern Customer Ops & Systems,10+ years,OWN,55000.0,Verified,Dec-13,n,debt_consolidation,Debt Consolidation,481xx,MI,22.87,0,Oct-86,0,14,0,36638,61.2,27,w,10018.90,10018.90,21251.04,21251.04,17031.10,4219.94,0.0,0.0,0.0,0.0,1,INDIVIDUAL,0,0.0,114834.0,59900.0,0
39695,10159584,12011200,9750,9750,9750.0,36 months,13.98,333.14,C,C1,Medical Assistant,1 year,RENT,26000.0,Not Verified,Dec-13,n,debt_consolidation,Debt Consilation,927xx,CA,25.12,0,Jan-07,0,12,0,7967,52.8,28,f,3710.96,3710.96,7994.83,7994.83,6039.04,1955.79,0.0,0.0,0.0,0.0,1,INDIVIDUAL,0,0.0,14123.0,15100.0,0
39696,10159498,1319523,12000,12000,12000.0,36 months,6.62,368.45,A,A2,MANAGER INFORMATION DELIVERY,10+ years,MORTGAGE,105000.0,Not Verified,Dec-13,n,debt_consolidation,UNIVERSAL CARD,060xx,CT,14.05,0,Mar-94,1,12,0,13168,21.6,22,w,4266.62,4266.62,8842.80,8842.80,7733.38,1109.42,0.0,0.0,0.0,0.0,1,INDIVIDUAL,0,0.0,267646.0,61100.0,0
39697,10139658,11991209,12000,12000,12000.0,36 months,13.53,407.40,B,B5,On road manager,10+ years,RENT,40000.0,Source Verified,Dec-13,n,debt_consolidation,Debt consolidation,871xx,NM,16.94,0,Oct-98,0,7,2,5572,68.8,32,w,0.00,0.00,13359.77,13359.77,11999.99,1359.78,0.0,0.0,0.0,0.0,1,INDIVIDUAL,0,15386.0,13605.0,8100.0,0
39698,10159548,12011167,15000,15000,15000.0,36 months,8.90,476.30,A,A5,aircraft maintenance engineer,2 years,MORTGAGE,63000.0,Not Verified,Dec-13,n,debt_consolidation,Pay off,334xx,FL,16.51,0,Mar-98,0,8,0,11431,74.2,29,w,5449.27,5449.27,11431.20,11431.20,9550.73,1880.47,0.0,0.0,0.0,0.0,1,INDIVIDUAL,0,1514.0,272492.0,15400.0,0


In [11]:
data['emp_length'] = data['emp_length'].str.replace('\D*\W* years*','').str.replace('< ','')
data['emp_length'].astype('int64')

39694     10
39695      1
39696     10
39697     10
39698      2
          ..
855964     8
855965    10
855966     5
855967     1
855968    10
Name: emp_length, Length: 788656, dtype: int64

In [12]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [13]:
le.fit(data['grade'])
print(list(le.classes_))
data.grade = le.transform(data['grade']) 

['A', 'B', 'C', 'D', 'E', 'F', 'G']


In [14]:
data.grade

39694     1
39695     2
39696     0
39697     1
39698     0
         ..
855964    1
855965    1
855966    3
855967    4
855968    1
Name: grade, Length: 788656, dtype: int64

In [15]:
le.fit(data['term'])
print(list(le.classes_))
data.term = le.transform(data['term']) 

[' 36 months', ' 60 months']


In [16]:
data.term

39694     0
39695     0
39696     0
39697     0
39698     0
         ..
855964    0
855965    0
855966    1
855967    1
855968    0
Name: term, Length: 788656, dtype: int64

In [17]:
le.fit(data['sub_grade'])
print(list(le.classes_))
data.sub_grade = le.transform(data['sub_grade']) 

['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5', 'G1', 'G2', 'G3', 'G4', 'G5']


In [18]:
data.sub_grade

39694      6
39695     10
39696      1
39697      9
39698      4
          ..
855964     9
855965     9
855966    16
855967    22
855968     9
Name: sub_grade, Length: 788656, dtype: int64

In [19]:
le.fit(data['home_ownership'])
print(list(le.classes_))
data.home_ownership = le.transform(data['home_ownership']) 

['ANY', 'MORTGAGE', 'NONE', 'OTHER', 'OWN', 'RENT']


In [20]:
data.home_ownership

39694     4
39695     5
39696     1
39697     5
39698     1
         ..
855964    5
855965    1
855966    5
855967    5
855968    5
Name: home_ownership, Length: 788656, dtype: int64

In [21]:
data.verification_status = data.verification_status.str.replace('Source Verified','Verified')
le.fit(data['verification_status'])
print(list(le.classes_))
data.verification_status = le.transform(data['verification_status']) 

['Not Verified', 'Verified']


In [22]:
data.verification_status

39694     1
39695     0
39696     0
39697     1
39698     0
         ..
855964    1
855965    1
855966    1
855967    1
855968    1
Name: verification_status, Length: 788656, dtype: int64

In [23]:
le.fit(data['pymnt_plan'])
print(list(le.classes_))
data.pymnt_plan = le.transform(data['pymnt_plan']) 

['n', 'y']


In [24]:
data.pymnt_plan 

39694     0
39695     0
39696     0
39697     0
39698     0
         ..
855964    0
855965    0
855966    0
855967    0
855968    0
Name: pymnt_plan, Length: 788656, dtype: int64

In [25]:
le.fit(data['purpose'])
print(list(le.classes_))
data.purpose = le.transform(data['purpose']) 

['car', 'credit_card', 'debt_consolidation', 'educational', 'home_improvement', 'house', 'major_purchase', 'medical', 'moving', 'other', 'renewable_energy', 'small_business', 'vacation', 'wedding']


In [26]:
data.purpose

39694     2
39695     2
39696     2
39697     2
39698     2
         ..
855964    2
855965    4
855966    2
855967    2
855968    1
Name: purpose, Length: 788656, dtype: int64

In [27]:
le.fit(data['application_type'])
print(list(le.classes_))
data.application_type = le.transform(data['application_type']) 

['INDIVIDUAL', 'JOINT']


In [28]:
data.application_type

39694     0
39695     0
39696     0
39697     0
39698     0
         ..
855964    0
855965    0
855966    0
855967    0
855968    0
Name: application_type, Length: 788656, dtype: int64

In [29]:
data = data.rename(columns={'default_ind':'approval'})

In [30]:
from datetime import datetime
data['issue_d'] = data['issue_d'].str.replace('-',' ')
data['issue_d'] = pd.to_datetime(data['issue_d'] , format ='%b %y')
data['earliest_cr_line'] = pd.to_datetime(data['earliest_cr_line'] , format ='%b-%y')
# data['issue_d'] = data['issue_d'].apply(lambda x: x.strftime('%m %y'))

In [31]:
data = data.set_index(['issue_d'])
data = data.sort_index()
data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,approval
issue_d,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2012-08-01,1476351,1734350,22400,22400,22100.0,1,14.09,522.26,1,9,Wal-Mart Stores Inc.,10,1,60000.0,1,0,2,Debt consolidation,010xx,MA,20.35,0,2000-07-01,0,16,0,17857,44.2,22,f,9272.46,9148.3,20869.150000,20589.62,13127.53,7741.62,0.0,0.00,0.0000,0.0,1,0,0,0.0,135926.0,40400.0,0
2012-08-01,1479477,1737962,12000,12000,11975.0,0,12.12,399.26,1,7,City of Atlantic city,10,5,61000.0,0,0,2,Personal,084xx,NJ,21.40,0,2006-07-01,0,13,0,12353,81.8,17,f,0.00,0.0,13584.980000,13556.68,12000.00,1584.98,0.0,0.00,0.0000,0.0,1,0,0,0.0,38192.0,15100.0,0
2012-08-01,1479621,1738125,3000,3000,3000.0,0,15.31,104.46,2,11,"mohawk valley orthopedics, pc",10,4,35000.0,0,0,2,credit card payment,120xx,NY,10.80,2,2000-11-01,3,12,0,2976,69.2,25,f,0.00,0.0,3740.506628,3740.51,3000.00,740.51,0.0,0.00,0.0000,0.0,1,0,0,0.0,8108.0,4300.0,0
2012-08-01,1479620,1738124,1200,1200,1200.0,0,12.12,39.93,1,7,Not Collected,0,5,30000.0,0,0,8,Moving,316xx,GA,14.62,0,1992-01-01,0,10,0,6490,40.6,24,f,0.00,0.0,1430.283834,1430.28,1200.00,230.28,0.0,0.00,0.0000,0.0,1,0,0,0.0,10855.0,16000.0,0
2012-08-01,1479604,1737906,16000,16000,16000.0,0,14.33,549.42,2,10,Department of social services,5,1,100000.0,0,0,2,consolidation,064xx,CT,13.08,3,1994-02-01,1,18,0,9126,58.5,36,f,0.00,0.0,10646.690000,10646.69,6700.96,2626.26,0.0,1319.47,13.0655,0.0,1,0,0,0.0,376021.0,15600.0,1


In [32]:
data.dtypes

id                                     int64
member_id                              int64
loan_amnt                              int64
funded_amnt                            int64
funded_amnt_inv                      float64
term                                   int64
int_rate                             float64
installment                          float64
grade                                  int64
sub_grade                              int64
emp_title                             object
emp_length                            object
home_ownership                         int64
annual_inc                           float64
verification_status                    int64
pymnt_plan                             int64
purpose                                int64
title                                 object
zip_code                              object
addr_state                            object
dti                                  float64
delinq_2yrs                            int64
earliest_c